In [ ]:
# SAPO Adaptive I/J Experiment Configuration
# Dynamically adjusts I/J ratio during training

# ============================================
# ADAPTIVE I/J CONFIGURATION
# ============================================
EXPERIMENT_NAME = 'sapo_daptive_ij'

# Total samples per round (I + J = 4)
TOTAL_SAMPLES = 4            # Total rollouts per round
INITIAL_J = 2                # Starting J value (middle of range)
INITIAL_I = TOTAL_SAMPLES - INITIAL_J  # Computed automatically

# Adaptive algorithm parameters
ADAPTIVE_IJ_ENABLED = True   # Enable adaptive I/J selection
ADAPTATION_RATE = 0.1        # Learning rate (alpha) for gradient updates
BASELINE_ALPHA = 0.95        # EMA smoothing for reward baseline

# ============================================
# TRAINING MODE: TESTING (default) or PRODUCTION
# ============================================
# TESTING MODE (default): Quick validation ~6 minutes
MAX_ROUNDS = 50              # Testing: 10 rounds to verify adaptive algorithm

# PRODUCTION MODE: Full training ~21 hours
# Uncomment line below for production run:
# MAX_ROUNDS = 2000          # Production: Full 2000 rounds for convergence

# ============================================
# FIXED SETTINGS (same for all experiments)
# ============================================
NUM_NODES = 5                # Run 5 nodes (1 coordinator + 4 workers)
MODEL_NAME = 'HuggingFaceTB/SmolLM-360M-Instruct'
NUM_GENERATIONS = 8          # G: Completions per question (like paper)
SEED = 42                    # For reproducibility


# Checkpoint Configuration
CHECKPOINT_INTERVAL = 100    # Save checkpoints every 100 rounds
MAX_STAGES = 1               # Stages per round (1=default)

# Optional: HuggingFace Token
HUGGINGFACE_TOKEN = None  # Set to your token or keep None

# ============================================
# DISPLAY CONFIGURATION
# ============================================
mode = "TESTING" if MAX_ROUNDS <= 20 else "PRODUCTION"
estimated_time = "~6 minutes" if MAX_ROUNDS <= 20 else "~21 hours"

print("="*60)
print(f"SAPO Adaptive I/J Experiment - {mode} MODE")
print("="*60)
print(f"✓ Mode: {mode}")
print(f"✓ Nodes: {NUM_NODES} (1 coordinator + 4 workers on single A100 80GB)")
print(f"✓ Model: {MODEL_NAME}")
print(f"✓ Experiment: {EXPERIMENT_NAME}")
print(f"✓ Max Rounds: {MAX_ROUNDS}")
print()
print("🔄 ADAPTIVE I/J CONFIGURATION:")
print(f"   Total Samples: {TOTAL_SAMPLES} (I + J)")
print(f"   Initial: I={INITIAL_I}, J={INITIAL_J}")
print(f"   Adaptation Rate (α): {ADAPTATION_RATE}")
print(f"   Baseline Smoothing: {BASELINE_ALPHA}")
print()
print(f"Expected VRAM: ~30 GB peak (4 workers train, coordinator doesn't)")
print(f"Expected Time: {estimated_time}")
print()

if mode == "TESTING":
    print("🧪 TESTING MODE ENABLED")
    print("   Quick validation run - verifies:")
    print("   ✓ Adaptive algorithm updates I/J each round")
    print("   ✓ Logs show gradient updates")
    print("   ✓ System handles changing rollout counts")
    print()
    print("   After validation succeeds, uncomment production line")
    print("   in this cell to run full 2000-round training.")
else:
    print("📊 PRODUCTION MODE - Adaptive I/J")
    print("   Expected behavior:")
    print("   - J starts at 2 (middle of range)")
    print("   - Increases if swarm helps (likely converges to J=4)")
    print("   - Decreases if local training is better")
    print("   - Should match/exceed best fixed ratio (Config 2)")
    print()
    print("   ⚠️  This is a THESIS CONTRIBUTION - novel algorithm!")

print("="*60)

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set base path (MUST BE SAME ACROSS ALL NODES)
GDRIVE_BASE_PATH = '/content/drive/MyDrive/rl-swarm'
os.makedirs(GDRIVE_BASE_PATH, exist_ok=True)

print(f"✓ Google Drive mounted at: {GDRIVE_BASE_PATH}")

## 3. System Setup & GPU Verification

In [ ]:
import torch

print("="*60)
print("GPU Verification")
print("="*60)

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"✓ GPU: {gpu_name}")
    print(f"✓ Total VRAM: {total_memory:.1f} GB")
    print()
    
    required_memory = (NUM_NODES - 1) * 6.5 + 4  # 4 workers + coordinator
    print(f"Memory Requirements:")
    print(f"  Workers: {NUM_NODES - 1} × 6.5 GB = {(NUM_NODES - 1) * 6.5:.1f} GB")
    print(f"  Coordinator: ~4 GB (non-training)")
    print(f"  Total estimated: {required_memory:.1f} GB")
    print(f"  Available: {total_memory:.1f} GB")
    print(f"  Margin: {total_memory - required_memory:.1f} GB")
    print()
    
    if total_memory < required_memory:
        raise RuntimeError(f"Insufficient GPU memory: need {required_memory:.0f} GB, have {total_memory:.1f} GB")
    else:
        print(f"✅ Sufficient VRAM for {NUM_NODES} nodes")
else:
    raise RuntimeError("No GPU detected! Select A100 GPU runtime: Runtime > Change runtime type > A100 GPU")

## 4. Clone Repository & Install Dependencies

In [ ]:
%cd /content

# Remove existing directory if it exists
if os.path.exists('/content/rl-swarm'):
    print("Removing existing repository...")
    !rm -rf /content/rl-swarm

# Clone fresh copy
print("Cloning repository...")
!git clone https://github.com/Elrashid/rl-swarm.git /content/rl-swarm

# Change to repo directory
%cd /content/rl-swarm

# Verify clone worked
if not os.path.exists('requirements.txt'):
    raise FileNotFoundError("Repository clone failed - requirements.txt not found")

print("✓ Repository cloned successfully")
print()

# Install dependencies
print("Installing dependencies (this may take 3-5 minutes)...")
!pip install -q -r requirements.txt
!pip install -q 'protobuf>=4.25.0,<5.0'

# Verify installation
try:
    import reasoning_gym
    from rgym_exp.src.adaptive_ij import GradientAdaptiveIJ
    print()
    print("✓ Dependencies installed successfully")
    print("✓ reasoning-gym verified")
    print("✓ Adaptive I/J algorithm verified")
except ImportError as e:
    print()
    print("❌ ERROR: Installation failed!")
    print(f"   {e}")
    raise

## 5. Initialize Experiment

In [ ]:
from rgym_exp.utils.experiment_manager import init_experiment

# Initialize experiment structure in Google Drive
config_overrides = {
    'training.max_round': MAX_ROUNDS,
    'training.num_generations': NUM_GENERATIONS,
    'training.total_samples': TOTAL_SAMPLES,
    'training.initial_I': INITIAL_I,
    'training.initial_J': INITIAL_J,
    'training.adaptive_ij_enabled': ADAPTIVE_IJ_ENABLED,
    'training.adaptation_rate': ADAPTATION_RATE,
    'training.seed': SEED,
}

init_experiment(
    gdrive_base_path=GDRIVE_BASE_PATH,
    experiment_name=EXPERIMENT_NAME,
    config_overrides=config_overrides
)

print(f"✓ Experiment initialized: {EXPERIMENT_NAME}")
print(f"  Path: {GDRIVE_BASE_PATH}/experiments/{EXPERIMENT_NAME}")
print(f"  Config: Initial I={INITIAL_I}, J={INITIAL_J} (total={TOTAL_SAMPLES})")
print(f"  Adaptive: Enabled with α={ADAPTATION_RATE}")
print()
print("🔄 I/J will adapt dynamically based on reward feedback!")

## 6. Launch 5-Node Adaptive Swarm (KEY CELL)

**This cell:**
1. Spawns 5 separate Python processes
2. Each process runs `swarm_launcher.py` with unique NODE_ID
3. All processes share GPU 0 (CUDA_VISIBLE_DEVICES=0)
4. Coordinator (node_0) manages round progression
5. Workers (node_1-4) follow coordinator

**Note for Adaptive:** All nodes use adaptive I/J algorithm - J will change each round based on performance. The algorithm automatically adjusts the ratio of local to external rollouts to maximize reward.

**Logs:** Each node writes to Google Drive at `{GDRIVE_BASE_PATH}/experiments/{EXPERIMENT_NAME}/logs/`

**Monitor:** Use next cell (Cell 7) to track progress in real-time

In [ ]:
i
m
p
o
r
t
 
s
u
b
p
r
o
c
e
s
s

i
m
p
o
r
t
 
o
s

i
m
p
o
r
t
 
t
i
m
e

f
r
o
m
 
d
a
t
e
t
i
m
e
 
i
m
p
o
r
t
 
d
a
t
e
t
i
m
e


m
o
d
e
_
l
a
b
e
l
 
=
 
"
T
E
S
T
I
N
G
"
 
i
f
 
M
A
X
_
R
O
U
N
D
S
 
<
=
 
2
0
 
e
l
s
e
 
"
P
R
O
D
U
C
T
I
O
N
"

e
s
t
i
m
a
t
e
d
_
d
u
r
a
t
i
o
n
 
=
 
"
~
6
 
m
i
n
u
t
e
s
"
 
i
f
 
M
A
X
_
R
O
U
N
D
S
 
<
=
 
2
0
 
e
l
s
e
 
"
~
2
1
 
h
o
u
r
s
"


p
r
i
n
t
(
"
=
"
*
6
0
)

p
r
i
n
t
(
f
"
L
a
u
n
c
h
i
n
g
 
{
N
U
M
_
N
O
D
E
S
}
-
N
o
d
e
 
A
d
a
p
t
i
v
e
 
S
w
a
r
m
 
(
{
m
o
d
e
_
l
a
b
e
l
}
 
M
O
D
E
)
"
)

p
r
i
n
t
(
"
=
"
*
6
0
)

p
r
i
n
t
(
f
"
E
x
p
e
r
i
m
e
n
t
:
 
{
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
}
"
)

p
r
i
n
t
(
f
"
M
o
d
e
l
:
 
{
M
O
D
E
L
_
N
A
M
E
}
"
)

p
r
i
n
t
(
f
"
I
n
i
t
i
a
l
 
C
o
n
f
i
g
:
 
I
=
{
I
N
I
T
I
A
L
_
I
}
,
 
J
=
{
I
N
I
T
I
A
L
_
J
}
,
 
G
=
{
N
U
M
_
G
E
N
E
R
A
T
I
O
N
S
}
"
)

p
r
i
n
t
(
f
"
A
d
a
p
t
a
t
i
o
n
:
 
α
=
{
A
D
A
P
T
A
T
I
O
N
_
R
A
T
E
}
 
(
w
i
l
l
 
a
d
j
u
s
t
 
I
/
J
 
e
a
c
h
 
r
o
u
n
d
)
"
)

p
r
i
n
t
(
f
"
R
o
u
n
d
s
:
 
{
M
A
X
_
R
O
U
N
D
S
}
 
(
{
e
s
t
i
m
a
t
e
d
_
d
u
r
a
t
i
o
n
}
)
"
)

p
r
i
n
t
(
f
"
H
a
r
d
w
a
r
e
:
 
A
l
l
 
{
N
U
M
_
N
O
D
E
S
}
 
n
o
d
e
s
 
o
n
 
s
i
n
g
l
e
 
G
P
U
 
(
A
1
0
0
 
8
0
G
B
)
"
)

p
r
i
n
t
(
"
=
"
*
6
0
)

p
r
i
n
t
(
)


#
 
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=

#
 
C
O
D
E
 
V
E
R
S
I
O
N
 
C
H
E
C
K

#
 
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=

p
r
i
n
t
(
"
C
h
e
c
k
i
n
g
 
c
o
d
e
 
v
e
r
s
i
o
n
.
.
.
"
)

r
e
s
u
l
t
 
=
 
s
u
b
p
r
o
c
e
s
s
.
r
u
n
(
[
'
g
i
t
'
,
 
'
l
o
g
'
,
 
'
-
-
o
n
e
l
i
n
e
'
,
 
'
-
5
'
]
,

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
a
p
t
u
r
e
_
o
u
t
p
u
t
=
T
r
u
e
,
 
t
e
x
t
=
T
r
u
e
,
 
c
w
d
=
'
/
c
o
n
t
e
n
t
/
r
l
-
s
w
a
r
m
'
)

c
o
m
m
i
t
s
 
=
 
r
e
s
u
l
t
.
s
t
d
o
u
t
.
s
t
r
i
p
(
)
.
s
p
l
i
t
(
'
\
n
'
)

l
a
t
e
s
t
_
c
o
m
m
i
t
 
=
 
c
o
m
m
i
t
s
[
0
]
 
i
f
 
c
o
m
m
i
t
s
 
e
l
s
e
 
"
u
n
k
n
o
w
n
"


p
r
i
n
t
(
f
"
L
a
t
e
s
t
 
c
o
m
m
i
t
:
 
{
l
a
t
e
s
t
_
c
o
m
m
i
t
}
"
)

p
r
i
n
t
(
f
"
R
e
c
e
n
t
 
c
o
m
m
i
t
s
:
"
)

f
o
r
 
c
o
m
m
i
t
 
i
n
 
c
o
m
m
i
t
s
[
:
3
]
:

 
 
 
 
p
r
i
n
t
(
f
"
 
 
{
c
o
m
m
i
t
}
"
)

p
r
i
n
t
(
)


#
 
C
h
e
c
k
 
f
o
r
 
r
o
l
l
o
u
t
 
p
u
b
l
i
s
h
i
n
g
 
f
i
x
 
(
c
r
i
t
i
c
a
l
 
f
o
r
 
a
d
a
p
t
i
v
e
 
-
 
n
e
e
d
s
 
r
o
l
l
o
u
t
 
s
h
a
r
i
n
g
!
)

h
a
s
_
r
o
l
l
o
u
t
_
f
i
x
 
=
 
a
n
y
(
'
0
2
7
f
b
2
d
'
 
i
n
 
c
o
m
m
i
t
 
o
r
 
'
r
o
l
l
o
u
t
 
p
u
b
l
i
s
h
i
n
g
'
 
i
n
 
c
o
m
m
i
t
.
l
o
w
e
r
(
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
c
o
m
m
i
t
 
i
n
 
c
o
m
m
i
t
s
)


i
f
 
h
a
s
_
r
o
l
l
o
u
t
_
f
i
x
:

 
 
 
 
p
r
i
n
t
(
"
✓
 
H
a
s
 
r
o
l
l
o
u
t
 
p
u
b
l
i
s
h
i
n
g
 
f
i
x
 
(
c
o
m
m
i
t
 
0
2
7
f
b
2
d
)
"
)

e
l
s
e
:

 
 
 
 
p
r
i
n
t
(
"
⚠
️
 
 
W
A
R
N
I
N
G
:
 
M
i
s
s
i
n
g
 
r
o
l
l
o
u
t
 
p
u
b
l
i
s
h
i
n
g
 
f
i
x
!
"
)

 
 
 
 
p
r
i
n
t
(
"
 
 
 
A
d
a
p
t
i
v
e
 
a
l
g
o
r
i
t
h
m
 
R
E
Q
U
I
R
E
S
 
r
o
l
l
o
u
t
 
s
h
a
r
i
n
g
 
t
o
 
w
o
r
k
!
"
)

 
 
 
 
p
r
i
n
t
(
"
 
 
 
E
x
p
e
c
t
e
d
:
 
c
o
m
m
i
t
 
0
2
7
f
b
2
d
 
'
f
i
x
:
 
I
m
p
l
e
m
e
n
t
 
p
r
o
p
e
r
 
r
o
l
l
o
u
t
 
p
u
b
l
i
s
h
i
n
g
'
"
)


p
r
i
n
t
(
)

p
r
i
n
t
(
"
=
"
*
6
0
)

p
r
i
n
t
(
"
S
t
a
r
t
i
n
g
 
n
o
d
e
 
p
r
o
c
e
s
s
e
s
.
.
.
"
)

p
r
i
n
t
(
"
=
"
*
6
0
)

p
r
i
n
t
(
)


p
r
o
c
e
s
s
e
s
 
=
 
[
]

p
r
o
c
e
s
s
_
s
t
d
e
r
r
 
=
 
[
]
 
 
#
 
S
t
o
r
e
 
s
t
d
e
r
r
 
f
o
r
 
e
a
c
h
 
p
r
o
c
e
s
s

s
t
a
r
t
_
t
i
m
e
 
=
 
t
i
m
e
.
t
i
m
e
(
)
 
 
#
 
F
o
r
 
E
T
A
 
c
a
l
c
u
l
a
t
i
o
n


f
o
r
 
n
o
d
e
_
i
d
 
i
n
 
r
a
n
g
e
(
N
U
M
_
N
O
D
E
S
)
:

 
 
 
 
#
 
E
n
v
i
r
o
n
m
e
n
t
 
v
a
r
i
a
b
l
e
s
 
f
o
r
 
t
h
i
s
 
n
o
d
e

 
 
 
 
e
n
v
 
=
 
o
s
.
e
n
v
i
r
o
n
.
c
o
p
y
(
)

 
 
 
 
e
n
v
[
'
N
O
D
E
_
I
D
'
]
 
=
 
f
'
n
o
d
e
_
{
n
o
d
e
_
i
d
}
'

 
 
 
 
e
n
v
[
'
N
O
D
E
_
R
O
L
E
'
]
 
=
 
'
c
o
o
r
d
i
n
a
t
o
r
'
 
i
f
 
n
o
d
e
_
i
d
 
=
=
 
0
 
e
l
s
e
 
'
w
o
r
k
e
r
'

 
 
 
 
e
n
v
[
'
M
O
D
E
L
_
N
A
M
E
'
]
 
=
 
M
O
D
E
L
_
N
A
M
E

 
 
 
 
e
n
v
[
'
N
U
M
_
T
R
A
I
N
_
S
A
M
P
L
E
S
'
]
 
=
 
s
t
r
(
I
N
I
T
I
A
L
_
I
)
 
 
#
 
S
t
a
r
t
i
n
g
 
I
 
v
a
l
u
e

 
 
 
 
e
n
v
[
'
N
U
M
_
T
R
A
N
S
P
L
A
N
T
_
T
R
E
E
S
'
]
 
=
 
s
t
r
(
I
N
I
T
I
A
L
_
J
)
 
 
#
 
S
t
a
r
t
i
n
g
 
J
 
v
a
l
u
e

 
 
 
 
e
n
v
[
'
N
U
M
_
G
E
N
E
R
A
T
I
O
N
S
'
]
 
=
 
s
t
r
(
N
U
M
_
G
E
N
E
R
A
T
I
O
N
S
)

 
 
 
 
e
n
v
[
'
M
A
X
_
R
O
U
N
D
S
'
]
 
=
 
s
t
r
(
M
A
X
_
R
O
U
N
D
S
)

 
 
 
 
e
n
v
[
'
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
'
]
 
=
 
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E

 
 
 
 
e
n
v
[
'
G
D
R
I
V
E
_
P
A
T
H
'
]
 
=
 
G
D
R
I
V
E
_
B
A
S
E
_
P
A
T
H

 
 
 
 
e
n
v
[
'
C
U
D
A
_
V
I
S
I
B
L
E
_
D
E
V
I
C
E
S
'
]
 
=
 
'
0
'
 
 
#
 
A
l
l
 
n
o
d
e
s
 
s
h
a
r
e
 
G
P
U
 
0

 
 
 
 
e
n
v
[
'
S
E
E
D
'
]
 
=
 
s
t
r
(
S
E
E
D
 
+
 
n
o
d
e
_
i
d
)
 
 
#
 
D
i
f
f
e
r
e
n
t
 
s
e
e
d
 
p
e
r
 
n
o
d
e
 
(
d
i
v
e
r
s
i
t
y
)


 
 
 
 
#
 
A
d
a
p
t
i
v
e
 
I
/
J
 
c
o
n
f
i
g
u
r
a
t
i
o
n

 
 
 
 
e
n
v
[
'
A
D
A
P
T
I
V
E
_
I
J
_
E
N
A
B
L
E
D
'
]
 
=
 
'
T
r
u
e
'

 
 
 
 
e
n
v
[
'
A
D
A
P
T
I
V
E
_
I
J
_
A
L
P
H
A
'
]
 
=
 
s
t
r
(
A
D
A
P
T
A
T
I
O
N
_
R
A
T
E
)

 
 
 
 
e
n
v
[
'
A
D
A
P
T
I
V
E
_
I
J
_
B
A
S
E
L
I
N
E
_
A
L
P
H
A
'
]
 
=
 
s
t
r
(
B
A
S
E
L
I
N
E
_
A
L
P
H
A
)

 
 
 
 
e
n
v
[
'
A
D
A
P
T
I
V
E
_
I
J
_
I
N
I
T
I
A
L
_
J
'
]
 
=
 
s
t
r
(
I
N
I
T
I
A
L
_
J
)


 
 
 
 
#
 
R
o
l
l
o
u
t
 
s
h
a
r
i
n
g
 
c
o
n
f
i
g
u
r
a
t
i
o
n

 
 
 
 
e
n
v
[
'
R
O
L
L
O
U
T
_
P
U
B
L
I
S
H
_
F
R
E
Q
U
E
N
C
Y
'
]
 
=
 
R
O
L
L
O
U
T
_
P
U
B
L
I
S
H
_
F
R
E
Q
U
E
N
C
Y

 
 
 
 
e
n
v
[
'
R
O
L
L
O
U
T
_
C
L
E
A
N
U
P
_
E
N
A
B
L
E
D
'
]
 
=
 
s
t
r
(
R
O
L
L
O
U
T
_
C
L
E
A
N
U
P
_
E
N
A
B
L
E
D
)

 
 
 
 
e
n
v
[
'
R
O
L
L
O
U
T
_
K
E
E
P
_
L
A
S
T
_
N
_
R
O
U
N
D
S
'
]
 
=
 
s
t
r
(
R
O
L
L
O
U
T
_
K
E
E
P
_
L
A
S
T
_
N
_
R
O
U
N
D
S
)

 
 
 
 
e
n
v
[
'
R
O
L
L
O
U
T
_
A
R
C
H
I
V
E
_
O
L
D
'
]
 
=
 
s
t
r
(
R
O
L
L
O
U
T
_
A
R
C
H
I
V
E
_
O
L
D
)

 
 
 
 
e
n
v
[
'
C
H
E
C
K
P
O
I
N
T
_
I
N
T
E
R
V
A
L
'
]
 
=
 
s
t
r
(
C
H
E
C
K
P
O
I
N
T
_
I
N
T
E
R
V
A
L
)

 
 
 
 
e
n
v
[
'
M
A
X
_
S
T
A
G
E
S
'
]
 
=
 
s
t
r
(
M
A
X
_
S
T
A
G
E
S
)


 
 
 
 
#
 
E
n
a
b
l
e
 
d
e
b
u
g
 
l
o
g
g
i
n
g
 
f
o
r
 
s
t
a
t
e
.
t
r
e
e
s
 
t
r
a
c
k
i
n
g

 
 
 
 
e
n
v
[
'
D
E
B
U
G
_
T
R
E
E
S
'
]
 
=
 
'
t
r
u
e
'


 
 
 
 
i
f
 
H
U
G
G
I
N
G
F
A
C
E
_
T
O
K
E
N
:

 
 
 
 
 
 
 
 
e
n
v
[
'
H
U
G
G
I
N
G
F
A
C
E
_
A
C
C
E
S
S
_
T
O
K
E
N
'
]
 
=
 
H
U
G
G
I
N
G
F
A
C
E
_
T
O
K
E
N


 
 
 
 
#
 
L
a
u
n
c
h
 
p
r
o
c
e
s
s
 
w
i
t
h
 
s
t
d
e
r
r
/
s
t
d
o
u
t
 
c
a
p
t
u
r
e

 
 
 
 
i
m
p
o
r
t
 
s
y
s

 
 
 
 
p
r
o
c
e
s
s
 
=
 
s
u
b
p
r
o
c
e
s
s
.
P
o
p
e
n
(

 
 
 
 
 
 
 
 
[
s
y
s
.
e
x
e
c
u
t
a
b
l
e
,
 
'
-
m
'
,
 
'
r
g
y
m
_
e
x
p
.
r
u
n
n
e
r
.
s
w
a
r
m
_
l
a
u
n
c
h
e
r
'
]
,

 
 
 
 
 
 
 
 
e
n
v
=
e
n
v
,

 
 
 
 
 
 
 
 
c
w
d
=
'
/
c
o
n
t
e
n
t
/
r
l
-
s
w
a
r
m
'
,

 
 
 
 
 
 
 
 
s
t
d
e
r
r
=
s
u
b
p
r
o
c
e
s
s
.
P
I
P
E
,
 
 
#
 
C
a
p
t
u
r
e
 
e
r
r
o
r
s

 
 
 
 
 
 
 
 
s
t
d
o
u
t
=
s
u
b
p
r
o
c
e
s
s
.
P
I
P
E
,
 
 
#
 
C
a
p
t
u
r
e
 
o
u
t
p
u
t

 
 
 
 
 
 
 
 
t
e
x
t
=
T
r
u
e

 
 
 
 
)

 
 
 
 
p
r
o
c
e
s
s
e
s
.
a
p
p
e
n
d
(
p
r
o
c
e
s
s
)

 
 
 
 
p
r
o
c
e
s
s
_
s
t
d
e
r
r
.
a
p
p
e
n
d
(
p
r
o
c
e
s
s
.
s
t
d
e
r
r
)


 
 
 
 
r
o
l
e
 
=
 
"
C
O
O
R
D
I
N
A
T
O
R
"
 
i
f
 
n
o
d
e
_
i
d
 
=
=
 
0
 
e
l
s
e
 
"
W
O
R
K
E
R
 
 
 
 
 
"

 
 
 
 
p
r
i
n
t
(
f
"
✓
 
S
t
a
r
t
e
d
 
n
o
d
e
_
{
n
o
d
e
_
i
d
}
 
(
{
r
o
l
e
}
)
 
-
 
P
I
D
:
 
{
p
r
o
c
e
s
s
.
p
i
d
:
5
d
}
"
)


 
 
 
 
#
 
S
t
a
g
g
e
r
 
s
t
a
r
t
u
p
:
 
c
o
o
r
d
i
n
a
t
o
r
 
g
e
t
s
 
1
0
s
,
 
w
o
r
k
e
r
s
 
g
e
t
 
5
s

 
 
 
 
d
e
l
a
y
 
=
 
1
0
 
i
f
 
n
o
d
e
_
i
d
 
=
=
 
0
 
e
l
s
e
 
5

 
 
 
 
t
i
m
e
.
s
l
e
e
p
(
d
e
l
a
y
)


p
r
i
n
t
(
)

p
r
i
n
t
(
"
=
"
*
6
0
)

p
r
i
n
t
(
"
P
R
O
C
E
S
S
 
H
E
A
L
T
H
 
C
H
E
C
K
"
)

p
r
i
n
t
(
"
=
"
*
6
0
)

p
r
i
n
t
(
"
W
a
i
t
i
n
g
 
3
0
 
s
e
c
o
n
d
s
 
f
o
r
 
p
r
o
c
e
s
s
e
s
 
t
o
 
i
n
i
t
i
a
l
i
z
e
.
.
.
"
)

t
i
m
e
.
s
l
e
e
p
(
3
0
)


c
r
a
s
h
e
d
_
n
o
d
e
s
 
=
 
[
]

f
o
r
 
i
,
 
p
 
i
n
 
e
n
u
m
e
r
a
t
e
(
p
r
o
c
e
s
s
e
s
)
:

 
 
 
 
r
e
t
u
r
n
c
o
d
e
 
=
 
p
.
p
o
l
l
(
)

 
 
 
 
i
f
 
r
e
t
u
r
n
c
o
d
e
 
i
s
 
n
o
t
 
N
o
n
e
:

 
 
 
 
 
 
 
 
c
r
a
s
h
e
d
_
n
o
d
e
s
.
a
p
p
e
n
d
(
i
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
❌
 
n
o
d
e
_
{
i
}
:
 
C
R
A
S
H
E
D
 
(
e
x
i
t
 
c
o
d
e
 
{
r
e
t
u
r
n
c
o
d
e
}
)
"
)

 
 
 
 
 
 
 
 
#
 
T
r
y
 
t
o
 
r
e
a
d
 
s
t
d
e
r
r

 
 
 
 
 
 
 
 
t
r
y
:

 
 
 
 
 
 
 
 
 
 
 
 
s
t
d
e
r
r
_
o
u
t
p
u
t
 
=
 
p
r
o
c
e
s
s
_
s
t
d
e
r
r
[
i
]
.
r
e
a
d
(
)

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
s
t
d
e
r
r
_
o
u
t
p
u
t
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
 
E
r
r
o
r
 
o
u
t
p
u
t
:
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
l
i
n
e
 
i
n
 
s
t
d
e
r
r
_
o
u
t
p
u
t
.
s
p
l
i
t
(
'
\
n
'
)
[
-
1
0
:
]
:
 
 
#
 
L
a
s
t
 
1
0
 
l
i
n
e
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
l
i
n
e
.
s
t
r
i
p
(
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
 
 
 
{
l
i
n
e
}
"
)

 
 
 
 
 
 
 
 
e
x
c
e
p
t
:

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
 
(
C
o
u
l
d
 
n
o
t
 
r
e
a
d
 
e
r
r
o
r
 
o
u
t
p
u
t
)
"
)

 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
✓
 
n
o
d
e
_
{
i
}
:
 
R
U
N
N
I
N
G
"
)


p
r
i
n
t
(
)


i
f
 
c
r
a
s
h
e
d
_
n
o
d
e
s
:

 
 
 
 
p
r
i
n
t
(
f
"
⚠
️
 
 
W
A
R
N
I
N
G
:
 
{
l
e
n
(
c
r
a
s
h
e
d
_
n
o
d
e
s
)
}
/
{
N
U
M
_
N
O
D
E
S
}
 
n
o
d
e
s
 
c
r
a
s
h
e
d
!
"
)

 
 
 
 
p
r
i
n
t
(
f
"
 
 
 
C
r
a
s
h
e
d
:
 
{
'
,
 
'
.
j
o
i
n
(
f
'
n
o
d
e
_
{
i
}
'
 
f
o
r
 
i
 
i
n
 
c
r
a
s
h
e
d
_
n
o
d
e
s
)
}
"
)

 
 
 
 
p
r
i
n
t
(
f
"
 
 
 
C
h
e
c
k
 
l
o
g
s
 
i
n
:
 
{
G
D
R
I
V
E
_
B
A
S
E
_
P
A
T
H
}
/
e
x
p
e
r
i
m
e
n
t
s
/
{
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
}
/
l
o
g
s
/
"
)

 
 
 
 
p
r
i
n
t
(
)

 
 
 
 
u
s
e
r
_
i
n
p
u
t
 
=
 
i
n
p
u
t
(
"
C
o
n
t
i
n
u
e
 
w
i
t
h
 
r
e
m
a
i
n
i
n
g
 
n
o
d
e
s
?
 
(
y
e
s
/
n
o
)
:
 
"
)

 
 
 
 
i
f
 
u
s
e
r
_
i
n
p
u
t
.
l
o
w
e
r
(
)
 
!
=
 
'
y
e
s
'
:

 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
T
e
r
m
i
n
a
t
i
n
g
 
a
l
l
 
p
r
o
c
e
s
s
e
s
.
.
.
"
)

 
 
 
 
 
 
 
 
f
o
r
 
p
 
i
n
 
p
r
o
c
e
s
s
e
s
:

 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
p
.
p
o
l
l
(
)
 
i
s
 
N
o
n
e
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
.
t
e
r
m
i
n
a
t
e
(
)

 
 
 
 
 
 
 
 
r
a
i
s
e
 
R
u
n
t
i
m
e
E
r
r
o
r
(
f
"
{
l
e
n
(
c
r
a
s
h
e
d
_
n
o
d
e
s
)
}
 
n
o
d
e
s
 
c
r
a
s
h
e
d
 
-
 
s
e
e
 
e
r
r
o
r
s
 
a
b
o
v
e
"
)

e
l
s
e
:

 
 
 
 
p
r
i
n
t
(
f
"
✅
 
A
l
l
 
{
N
U
M
_
N
O
D
E
S
}
 
n
o
d
e
s
 
l
a
u
n
c
h
e
d
 
s
u
c
c
e
s
s
f
u
l
l
y
!
"
)


p
r
i
n
t
(
)

p
r
i
n
t
(
f
"
✓
 
T
r
a
i
n
i
n
g
 
w
i
l
l
 
r
u
n
 
f
o
r
 
{
e
s
t
i
m
a
t
e
d
_
d
u
r
a
t
i
o
n
}
 
(
{
M
A
X
_
R
O
U
N
D
S
}
 
r
o
u
n
d
s
)
"
)

p
r
i
n
t
(
f
"
✓
 
I
/
J
 
w
i
l
l
 
a
d
a
p
t
 
e
a
c
h
 
r
o
u
n
d
 
b
a
s
e
d
 
o
n
 
r
e
w
a
r
d
"
)

p
r
i
n
t
(
f
"
✓
 
L
o
g
s
 
l
o
c
a
t
i
o
n
:
 
{
G
D
R
I
V
E
_
B
A
S
E
_
P
A
T
H
}
/
e
x
p
e
r
i
m
e
n
t
s
/
{
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
}
/
l
o
g
s
/
"
)

p
r
i
n
t
(
f
"
✓
 
C
h
e
c
k
p
o
i
n
t
s
:
 
E
v
e
r
y
 
{
C
H
E
C
K
P
O
I
N
T
_
I
N
T
E
R
V
A
L
}
 
r
o
u
n
d
s
"
)

p
r
i
n
t
(
)

i
f
 
M
A
X
_
R
O
U
N
D
S
 
<
=
 
2
0
:

 
 
 
 
p
r
i
n
t
(
"
🧪
 
T
E
S
T
I
N
G
 
M
O
D
E
:
 
A
f
t
e
r
 
c
o
m
p
l
e
t
i
o
n
,
 
c
h
e
c
k
 
C
e
l
l
 
8
 
f
o
r
 
r
e
s
u
l
t
s
"
)

 
 
 
 
p
r
i
n
t
(
"
 
 
 
T
h
e
n
 
u
n
c
o
m
m
e
n
t
 
p
r
o
d
u
c
t
i
o
n
 
l
i
n
e
 
i
n
 
C
e
l
l
 
2
 
f
o
r
 
f
u
l
l
 
t
r
a
i
n
i
n
g
"
)

e
l
s
e
:

 
 
 
 
p
r
i
n
t
(
"
⚠
️
 
 
K
e
e
p
 
t
h
i
s
 
n
o
t
e
b
o
o
k
 
o
p
e
n
 
(
b
r
o
w
s
e
r
 
t
a
b
 
a
c
t
i
v
e
)
"
)

 
 
 
 
p
r
i
n
t
(
"
⚠
️
 
 
C
o
l
a
b
 
m
a
y
 
d
i
s
c
o
n
n
e
c
t
 
a
f
t
e
r
 
1
2
-
2
4
 
h
o
u
r
s
"
)

 
 
 
 
p
r
i
n
t
(
"
⚠
️
 
 
T
r
a
i
n
i
n
g
 
w
i
l
l
 
c
o
n
t
i
n
u
e
 
-
 
u
s
e
 
C
e
l
l
 
7
.
5
 
t
o
 
c
h
e
c
k
 
p
r
o
g
r
e
s
s
 
a
f
t
e
r
 
r
e
c
o
n
n
e
c
t
"
)

p
r
i
n
t
(
)

p
r
i
n
t
(
"
M
o
n
i
t
o
r
 
a
d
a
p
t
i
v
e
 
b
e
h
a
v
i
o
r
 
i
n
 
C
e
l
l
 
7
 
b
e
l
o
w
.
.
.
"
)

## 7. Monitor Adaptive Training Progress

**Watch how I/J adapts in real-time!**

In [ ]:
i
m
p
o
r
t
 
t
i
m
e

f
r
o
m
 
I
P
y
t
h
o
n
.
d
i
s
p
l
a
y
 
i
m
p
o
r
t
 
c
l
e
a
r
_
o
u
t
p
u
t

i
m
p
o
r
t
 
p
a
n
d
a
s
 
a
s
 
p
d


p
r
i
n
t
(
"
S
t
a
r
t
i
n
g
 
t
r
a
i
n
i
n
g
 
m
o
n
i
t
o
r
.
.
.
"
)

p
r
i
n
t
(
"
P
r
e
s
s
 
'
S
t
o
p
'
 
b
u
t
t
o
n
 
o
r
 
C
t
r
l
+
C
 
t
o
 
i
n
t
e
r
r
u
p
t
\
n
"
)


m
o
n
i
t
o
r
_
s
t
a
r
t
_
t
i
m
e
 
=
 
t
i
m
e
.
t
i
m
e
(
)


t
r
y
:

 
 
 
 
w
h
i
l
e
 
T
r
u
e
:

 
 
 
 
 
 
 
 
c
l
e
a
r
_
o
u
t
p
u
t
(
w
a
i
t
=
T
r
u
e
)


 
 
 
 
 
 
 
 
#
 
C
h
e
c
k
 
p
r
o
c
e
s
s
 
s
t
a
t
u
s

 
 
 
 
 
 
 
 
r
u
n
n
i
n
g
 
=
 
s
u
m
(
1
 
f
o
r
 
p
 
i
n
 
p
r
o
c
e
s
s
e
s
 
i
f
 
p
.
p
o
l
l
(
)
 
i
s
 
N
o
n
e
)

 
 
 
 
 
 
 
 
c
o
m
p
l
e
t
e
d
 
=
 
N
U
M
_
N
O
D
E
S
 
-
 
r
u
n
n
i
n
g


 
 
 
 
 
 
 
 
c
u
r
r
e
n
t
_
t
i
m
e
 
=
 
d
a
t
e
t
i
m
e
.
n
o
w
(
)
.
s
t
r
f
t
i
m
e
(
'
%
Y
-
%
m
-
%
d
 
%
H
:
%
M
:
%
S
'
)

 
 
 
 
 
 
 
 
e
l
a
p
s
e
d
_
h
o
u
r
s
 
=
 
(
t
i
m
e
.
t
i
m
e
(
)
 
-
 
s
t
a
r
t
_
t
i
m
e
)
 
/
 
3
6
0
0


 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
=
"
*
7
0
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
S
A
P
O
 
T
r
a
i
n
i
n
g
 
M
o
n
i
t
o
r
 
-
 
{
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
}
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
T
i
m
e
:
 
{
c
u
r
r
e
n
t
_
t
i
m
e
}
 
|
 
E
l
a
p
s
e
d
:
 
{
e
l
a
p
s
e
d
_
h
o
u
r
s
:
.
1
f
}
h
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
=
"
*
7
0
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
)


 
 
 
 
 
 
 
 
#
 
N
o
d
e
 
s
t
a
t
u
s

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
N
o
d
e
s
:
 
{
r
u
n
n
i
n
g
}
/
{
N
U
M
_
N
O
D
E
S
}
 
r
u
n
n
i
n
g
,
 
{
c
o
m
p
l
e
t
e
d
}
 
c
o
m
p
l
e
t
e
d
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
)


 
 
 
 
 
 
 
 
#
 
G
P
U
 
m
e
m
o
r
y

 
 
 
 
 
 
 
 
i
f
 
t
o
r
c
h
.
c
u
d
a
.
i
s
_
a
v
a
i
l
a
b
l
e
(
)
:

 
 
 
 
 
 
 
 
 
 
 
 
r
e
s
e
r
v
e
d
 
=
 
t
o
r
c
h
.
c
u
d
a
.
m
e
m
o
r
y
_
r
e
s
e
r
v
e
d
(
0
)
 
/
 
1
e
9

 
 
 
 
 
 
 
 
 
 
 
 
t
o
t
a
l
 
=
 
t
o
r
c
h
.
c
u
d
a
.
g
e
t
_
d
e
v
i
c
e
_
p
r
o
p
e
r
t
i
e
s
(
0
)
.
t
o
t
a
l
_
m
e
m
o
r
y
 
/
 
1
e
9

 
 
 
 
 
 
 
 
 
 
 
 
u
t
i
l
i
z
a
t
i
o
n
 
=
 
(
r
e
s
e
r
v
e
d
 
/
 
t
o
t
a
l
)
 
*
 
1
0
0


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
G
P
U
:
 
{
r
e
s
e
r
v
e
d
:
.
1
f
}
 
/
 
{
t
o
t
a
l
:
.
1
f
}
 
G
B
 
(
{
u
t
i
l
i
z
a
t
i
o
n
:
.
1
f
}
%
)
"
)


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
u
t
i
l
i
z
a
t
i
o
n
 
>
 
9
0
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
⚠
️
 
 
W
A
R
N
I
N
G
:
 
H
i
g
h
 
m
e
m
o
r
y
 
u
s
a
g
e
!
"
)

 
 
 
 
 
 
 
 
 
 
 
 
e
l
i
f
 
u
t
i
l
i
z
a
t
i
o
n
 
>
 
7
5
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
⚠
️
 
 
M
e
m
o
r
y
 
u
s
a
g
e
 
e
l
e
v
a
t
e
d
"
)

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
)


 
 
 
 
 
 
 
 
#
 
T
r
a
i
n
i
n
g
 
p
r
o
g
r
e
s
s

 
 
 
 
 
 
 
 
t
r
y
:

 
 
 
 
 
 
 
 
 
 
 
 
f
r
o
m
 
r
g
y
m
_
e
x
p
.
u
t
i
l
s
.
e
x
p
e
r
i
m
e
n
t
_
m
a
n
a
g
e
r
 
i
m
p
o
r
t
 
g
e
t
_
e
x
p
e
r
i
m
e
n
t
_
s
t
a
t
u
s

 
 
 
 
 
 
 
 
 
 
 
 
s
t
a
t
u
s
 
=
 
g
e
t
_
e
x
p
e
r
i
m
e
n
t
_
s
t
a
t
u
s
(
G
D
R
I
V
E
_
B
A
S
E
_
P
A
T
H
,
 
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
)


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
s
t
a
t
u
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
u
r
r
e
n
t
_
r
o
u
n
d
 
=
 
s
t
a
t
u
s
.
g
e
t
(
'
c
u
r
r
e
n
t
_
r
o
u
n
d
'
,
 
0
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
o
g
r
e
s
s
_
p
c
t
 
=
 
(
c
u
r
r
e
n
t
_
r
o
u
n
d
 
/
 
M
A
X
_
R
O
U
N
D
S
)
 
*
 
1
0
0


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
P
r
o
g
r
e
s
s
:
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
R
o
u
n
d
:
 
{
c
u
r
r
e
n
t
_
r
o
u
n
d
:
4
d
}
 
/
 
{
M
A
X
_
R
O
U
N
D
S
}
 
(
{
p
r
o
g
r
e
s
s
_
p
c
t
:
5
.
1
f
}
%
)
"
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
P
r
o
g
r
e
s
s
 
b
a
r

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
a
r
_
l
e
n
g
t
h
 
=
 
5
0

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
i
l
l
e
d
 
=
 
i
n
t
(
b
a
r
_
l
e
n
g
t
h
 
*
 
p
r
o
g
r
e
s
s
_
p
c
t
 
/
 
1
0
0
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
a
r
 
=
 
'
█
'
 
*
 
f
i
l
l
e
d
 
+
 
'
░
'
 
*
 
(
b
a
r
_
l
e
n
g
t
h
 
-
 
f
i
l
l
e
d
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
[
{
b
a
r
}
]
"
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
E
T
A

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
c
u
r
r
e
n
t
_
r
o
u
n
d
 
>
 
1
0
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
h
o
u
r
s
_
p
e
r
_
r
o
u
n
d
 
=
 
e
l
a
p
s
e
d
_
h
o
u
r
s
 
/
 
c
u
r
r
e
n
t
_
r
o
u
n
d

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
e
m
a
i
n
i
n
g
_
r
o
u
n
d
s
 
=
 
M
A
X
_
R
O
U
N
D
S
 
-
 
c
u
r
r
e
n
t
_
r
o
u
n
d

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
t
a
_
h
o
u
r
s
 
=
 
r
e
m
a
i
n
i
n
g
_
r
o
u
n
d
s
 
*
 
h
o
u
r
s
_
p
e
r
_
r
o
u
n
d

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
E
T
A
:
 
{
e
t
a
_
h
o
u
r
s
:
.
1
f
}
 
h
o
u
r
s
 
(
~
{
e
t
a
_
h
o
u
r
s
/
2
4
:
.
1
f
}
 
d
a
y
s
)
"
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
T
O
T
A
L
 
C
U
M
U
L
A
T
I
V
E
 
R
E
W
A
R
D
 
(
P
R
I
M
A
R
Y
 
M
E
T
R
I
C
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=
=

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
'
t
o
t
a
l
_
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
'
 
i
n
 
s
t
a
t
u
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
 
=
 
s
t
a
t
u
s
.
g
e
t
(
'
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
'
,
 
{
}
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
o
t
a
l
_
c
u
m
u
l
a
t
i
v
e
 
=
 
s
t
a
t
u
s
[
'
t
o
t
a
l
_
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
'
]


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
┌
"
 
+
 
"
─
"
*
6
8
 
+
 
"
┐
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
│
"
 
+
 
"
 
T
O
T
A
L
 
C
U
M
U
L
A
T
I
V
E
 
R
E
W
A
R
D
 
(
S
A
P
O
 
P
a
p
e
r
 
M
e
t
r
i
c
)
"
.
c
e
n
t
e
r
(
6
8
)
 
+
 
"
│
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
├
"
 
+
 
"
─
"
*
6
8
 
+
 
"
┤
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
│
 
 
T
O
T
A
L
:
 
{
t
o
t
a
l
_
c
u
m
u
l
a
t
i
v
e
:
8
.
2
f
}
"
.
l
j
u
s
t
(
6
9
)
 
+
 
"
│
"
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
│
"
.
l
j
u
s
t
(
6
9
)
 
+
 
"
│
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
│
 
 
P
e
r
-
N
o
d
e
 
B
r
e
a
k
d
o
w
n
:
"
.
l
j
u
s
t
(
6
9
)
 
+
 
"
│
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
n
o
d
e
_
i
d
 
i
n
 
s
o
r
t
e
d
(
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
.
k
e
y
s
(
)
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
n
o
d
e
_
r
e
w
a
r
d
 
=
 
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
[
n
o
d
e
_
i
d
]

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
c
t
 
=
 
(
n
o
d
e
_
r
e
w
a
r
d
 
/
 
t
o
t
a
l
_
c
u
m
u
l
a
t
i
v
e
 
*
 
1
0
0
)
 
i
f
 
t
o
t
a
l
_
c
u
m
u
l
a
t
i
v
e
 
>
 
0
 
e
l
s
e
 
0

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
i
n
e
 
=
 
f
"
│
 
 
 
 
{
n
o
d
e
_
i
d
:
1
0
s
}
:
 
{
n
o
d
e
_
r
e
w
a
r
d
:
7
.
2
f
}
 
(
{
p
c
t
:
5
.
1
f
}
%
)
"

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
l
i
n
e
.
l
j
u
s
t
(
6
9
)
 
+
 
"
│
"
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
A
v
e
r
a
g
e
 
p
e
r
 
r
o
u
n
d

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
a
v
g
_
p
e
r
_
r
o
u
n
d
 
=
 
t
o
t
a
l
_
c
u
m
u
l
a
t
i
v
e
 
/
 
c
u
r
r
e
n
t
_
r
o
u
n
d
 
i
f
 
c
u
r
r
e
n
t
_
r
o
u
n
d
 
>
 
0
 
e
l
s
e
 
0

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
│
"
.
l
j
u
s
t
(
6
9
)
 
+
 
"
│
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
│
 
 
A
v
g
 
p
e
r
 
r
o
u
n
d
:
 
{
a
v
g
_
p
e
r
_
r
o
u
n
d
:
6
.
4
f
}
"
.
l
j
u
s
t
(
6
9
)
 
+
 
"
│
"
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
C
o
m
p
a
r
i
s
o
n
 
t
o
 
p
a
p
e
r
 
b
e
n
c
h
m
a
r
k
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
│
"
.
l
j
u
s
t
(
6
9
)
 
+
 
"
│
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
│
 
 
P
a
p
e
r
 
B
e
n
c
h
m
a
r
k
s
 
(
Q
w
e
n
2
.
5
,
 
2
0
0
0
 
r
o
u
n
d
s
)
:
"
.
l
j
u
s
t
(
6
9
)
 
+
 
"
│
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
│
 
 
 
 
B
a
s
e
l
i
n
e
 
(
8
/
0
)
:
 
 
 
 
5
6
2
"
.
l
j
u
s
t
(
6
9
)
 
+
 
"
│
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
│
 
 
 
 
C
o
n
f
i
g
 
2
 
(
4
/
4
)
:
 
 
1
,
0
9
3
 
(
+
9
4
%
)
 
⭐
"
.
l
j
u
s
t
(
6
9
)
 
+
 
"
│
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
└
"
 
+
 
"
─
"
*
6
8
 
+
 
"
┘
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
R
e
c
e
n
t
 
p
e
r
f
o
r
m
a
n
c
e

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
r
y
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
r
o
m
 
r
g
y
m
_
e
x
p
.
u
t
i
l
s
.
e
x
p
e
r
i
m
e
n
t
_
m
a
n
a
g
e
r
 
i
m
p
o
r
t
 
g
e
t
_
e
x
p
e
r
i
m
e
n
t
_
m
e
t
r
i
c
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
d
f
 
=
 
g
e
t
_
e
x
p
e
r
i
m
e
n
t
_
m
e
t
r
i
c
s
(
G
D
R
I
V
E
_
B
A
S
E
_
P
A
T
H
,
 
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
d
f
.
e
m
p
t
y
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
e
c
e
n
t
_
r
e
w
a
r
d
 
=
 
d
f
.
t
a
i
l
(
1
0
)
[
'
m
y
_
r
e
w
a
r
d
'
]
.
m
e
a
n
(
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
R
e
c
e
n
t
 
P
e
r
f
o
r
m
a
n
c
e
:
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
A
v
g
 
r
e
w
a
r
d
:
 
{
r
e
c
e
n
t
_
r
e
w
a
r
d
:
6
.
4
f
}
 
(
l
a
s
t
 
1
0
 
r
o
u
n
d
s
)
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
E
x
c
e
p
t
i
o
n
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
a
s
s


 
 
 
 
 
 
 
 
e
x
c
e
p
t
 
E
x
c
e
p
t
i
o
n
 
a
s
 
e
:

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
P
r
o
g
r
e
s
s
:
 
U
n
a
b
l
e
 
t
o
 
l
o
a
d
 
s
t
a
t
u
s
 
(
{
e
}
)
"
)

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
)


 
 
 
 
 
 
 
 
#
 
I
n
s
t
r
u
c
t
i
o
n
s

 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
-
"
*
7
0
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
P
r
e
s
s
 
'
S
t
o
p
'
 
b
u
t
t
o
n
 
o
r
 
C
t
r
l
+
C
 
t
o
 
h
a
l
t
 
t
r
a
i
n
i
n
g
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
N
e
x
t
 
u
p
d
a
t
e
 
i
n
 
6
0
 
s
e
c
o
n
d
s
.
.
.
"
)


 
 
 
 
 
 
 
 
#
 
E
x
i
t
 
i
f
 
a
l
l
 
c
o
m
p
l
e
t
e
d

 
 
 
 
 
 
 
 
i
f
 
r
u
n
n
i
n
g
 
=
=
 
0
:

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
)

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
=
"
*
7
0
)

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
✅
 
A
l
l
 
n
o
d
e
s
 
c
o
m
p
l
e
t
e
d
 
s
u
c
c
e
s
s
f
u
l
l
y
!
"
)

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
=
"
*
7
0
)

 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k


 
 
 
 
 
 
 
 
t
i
m
e
.
s
l
e
e
p
(
6
0
)


e
x
c
e
p
t
 
K
e
y
b
o
a
r
d
I
n
t
e
r
r
u
p
t
:

 
 
 
 
p
r
i
n
t
(
"
\
n
"
 
+
 
"
=
"
*
7
0
)

 
 
 
 
p
r
i
n
t
(
"
⚠
️
 
 
T
r
a
i
n
i
n
g
 
i
n
t
e
r
r
u
p
t
e
d
 
b
y
 
u
s
e
r
"
)

 
 
 
 
p
r
i
n
t
(
"
=
"
*
7
0
)

 
 
 
 
p
r
i
n
t
(
"
\
n
T
e
r
m
i
n
a
t
i
n
g
 
a
l
l
 
n
o
d
e
 
p
r
o
c
e
s
s
e
s
.
.
.
"
)


 
 
 
 
f
o
r
 
i
,
 
p
 
i
n
 
e
n
u
m
e
r
a
t
e
(
p
r
o
c
e
s
s
e
s
)
:

 
 
 
 
 
 
 
 
i
f
 
p
.
p
o
l
l
(
)
 
i
s
 
N
o
n
e
:

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
S
t
o
p
p
i
n
g
 
n
o
d
e
_
{
i
}
.
.
.
 
(
P
I
D
:
 
{
p
.
p
i
d
}
)
"
)

 
 
 
 
 
 
 
 
 
 
 
 
p
.
t
e
r
m
i
n
a
t
e
(
)


 
 
 
 
t
i
m
e
.
s
l
e
e
p
(
5
)


 
 
 
 
f
o
r
 
i
,
 
p
 
i
n
 
e
n
u
m
e
r
a
t
e
(
p
r
o
c
e
s
s
e
s
)
:

 
 
 
 
 
 
 
 
i
f
 
p
.
p
o
l
l
(
)
 
i
s
 
N
o
n
e
:

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
F
o
r
c
e
 
k
i
l
l
i
n
g
 
n
o
d
e
_
{
i
}
.
.
.
 
(
P
I
D
:
 
{
p
.
p
i
d
}
)
"
)

 
 
 
 
 
 
 
 
 
 
 
 
p
.
k
i
l
l
(
)


 
 
 
 
p
r
i
n
t
(
"
\
n
✓
 
A
l
l
 
p
r
o
c
e
s
s
e
s
 
t
e
r
m
i
n
a
t
e
d
"
)

 
 
 
 
p
r
i
n
t
(
"
\
n
💾
 
N
o
t
e
:
 
T
r
a
i
n
i
n
g
 
s
t
a
t
e
 
i
s
 
c
h
e
c
k
p
o
i
n
t
e
d
.
"
)

 
 
 
 
p
r
i
n
t
(
"
 
 
 
R
e
-
r
u
n
 
t
h
i
s
 
n
o
t
e
b
o
o
k
 
t
o
 
r
e
s
u
m
e
 
f
r
o
m
 
l
a
s
t
 
c
h
e
c
k
p
o
i
n
t
.
"
)

## 7.5. Check Real-Time Progress from GDrive (Optional)

**Reconnected after disconnect?** Run this cell to check training progress:
- Shows current round for each node
- Displays elapsed time and adaptive I/J values
- Works even if your notebook disconnected

Progress is saved to GDrive every round, logs flush every 30 seconds.

In [ ]:
#
 
=
=
=
 
R
e
a
l
-
T
i
m
e
 
P
r
o
g
r
e
s
s
 
V
i
e
w
e
r
 
=
=
=

#
 
R
u
n
 
t
h
i
s
 
c
e
l
l
 
a
n
y
t
i
m
e
 
t
o
 
c
h
e
c
k
 
p
r
o
g
r
e
s
s
 
f
r
o
m
 
G
D
r
i
v
e

#
 
U
s
e
f
u
l
 
i
f
 
y
o
u
 
r
e
c
o
n
n
e
c
t
 
a
f
t
e
r
 
n
o
t
e
b
o
o
k
 
d
i
s
c
o
n
n
e
c
t


i
m
p
o
r
t
 
s
y
s

s
y
s
.
p
a
t
h
.
a
p
p
e
n
d
(
'
/
c
o
n
t
e
n
t
/
r
l
-
s
w
a
r
m
'
)


f
r
o
m
 
r
g
y
m
_
e
x
p
.
u
t
i
l
s
.
e
x
p
e
r
i
m
e
n
t
_
m
a
n
a
g
e
r
 
i
m
p
o
r
t
 
g
e
t
_
e
x
p
e
r
i
m
e
n
t
_
s
t
a
t
u
s

f
r
o
m
 
r
g
y
m
_
e
x
p
.
u
t
i
l
s
.
p
r
o
g
r
e
s
s
_
t
r
a
c
k
e
r
 
i
m
p
o
r
t
 
g
e
t
_
e
x
p
e
r
i
m
e
n
t
_
p
r
o
g
r
e
s
s


#
 
G
e
t
 
c
o
m
p
r
e
h
e
n
s
i
v
e
 
s
t
a
t
u
s
 
i
n
c
l
u
d
i
n
g
 
c
u
m
u
l
a
t
i
v
e
 
r
e
w
a
r
d
s

s
t
a
t
u
s
 
=
 
g
e
t
_
e
x
p
e
r
i
m
e
n
t
_
s
t
a
t
u
s
(
G
D
R
I
V
E
_
B
A
S
E
_
P
A
T
H
,
 
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
)

p
r
o
g
r
e
s
s
 
=
 
g
e
t
_
e
x
p
e
r
i
m
e
n
t
_
p
r
o
g
r
e
s
s
(
G
D
R
I
V
E
_
B
A
S
E
_
P
A
T
H
,
 
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
)


p
r
i
n
t
(
"
=
"
*
7
0
)

p
r
i
n
t
(
"
R
E
A
L
-
T
I
M
E
 
P
R
O
G
R
E
S
S
 
F
R
O
M
 
G
D
R
I
V
E
 
(
A
D
A
P
T
I
V
E
 
I
/
J
)
"
)

p
r
i
n
t
(
"
=
"
*
7
0
)

p
r
i
n
t
(
f
"
E
x
p
e
r
i
m
e
n
t
:
 
{
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
}
"
)

p
r
i
n
t
(
f
"
R
o
u
n
d
:
 
{
s
t
a
t
u
s
.
g
e
t
(
'
c
u
r
r
e
n
t
_
r
o
u
n
d
'
,
 
0
)
}
 
/
 
{
M
A
X
_
R
O
U
N
D
S
}
"
)

p
r
i
n
t
(
f
"
A
c
t
i
v
e
 
p
e
e
r
s
:
 
{
s
t
a
t
u
s
.
g
e
t
(
'
a
c
t
i
v
e
_
p
e
e
r
s
'
,
 
0
)
}
"
)

p
r
i
n
t
(
)


#
 
C
u
m
u
l
a
t
i
v
e
 
R
e
w
a
r
d
s
 
(
S
A
P
O
 
p
a
p
e
r
 
m
e
t
r
i
c
)

i
f
 
'
t
o
t
a
l
_
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
'
 
i
n
 
s
t
a
t
u
s
:

 
 
 
 
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
 
=
 
s
t
a
t
u
s
.
g
e
t
(
'
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
'
,
 
{
}
)

 
 
 
 
t
o
t
a
l
_
c
u
m
u
l
a
t
i
v
e
 
=
 
s
t
a
t
u
s
[
'
t
o
t
a
l
_
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
'
]

 
 
 
 

 
 
 
 
p
r
i
n
t
(
"
C
u
m
u
l
a
t
i
v
e
 
R
e
w
a
r
d
s
 
(
A
D
A
P
T
I
V
E
 
-
 
S
A
P
O
 
p
a
p
e
r
 
m
e
t
r
i
c
)
:
"
)

 
 
 
 
p
r
i
n
t
(
f
"
 
 
T
o
t
a
l
 
(
a
l
l
 
n
o
d
e
s
)
:
 
{
t
o
t
a
l
_
c
u
m
u
l
a
t
i
v
e
:
7
.
2
f
}
"
)

 
 
 
 
i
f
 
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
:

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
P
e
r
 
n
o
d
e
:
"
)

 
 
 
 
 
 
 
 
f
o
r
 
n
o
d
e
_
i
d
 
i
n
 
s
o
r
t
e
d
(
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
.
k
e
y
s
(
)
)
:

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
 
 
{
n
o
d
e
_
i
d
:
1
0
s
}
:
 
{
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
[
n
o
d
e
_
i
d
]
:
7
.
2
f
}
"
)

 
 
 
 
p
r
i
n
t
(
)

e
l
s
e
:

 
 
 
 
p
r
i
n
t
(
"
C
u
m
u
l
a
t
i
v
e
 
r
e
w
a
r
d
s
:
 
N
o
t
 
a
v
a
i
l
a
b
l
e
 
y
e
t
 
(
t
r
a
i
n
i
n
g
 
j
u
s
t
 
s
t
a
r
t
e
d
)
"
)

 
 
 
 
p
r
i
n
t
(
)


#
 
P
e
r
-
n
o
d
e
 
p
r
o
g
r
e
s
s
 
d
e
t
a
i
l
s

p
r
i
n
t
(
"
P
e
r
-
N
o
d
e
 
P
r
o
g
r
e
s
s
:
"
)

f
o
r
 
n
o
d
e
_
i
d
,
 
n
o
d
e
_
d
a
t
a
 
i
n
 
p
r
o
g
r
e
s
s
.
g
e
t
(
'
n
o
d
e
s
'
,
 
{
}
)
.
i
t
e
m
s
(
)
:

 
 
 
 
i
f
 
'
e
r
r
o
r
'
 
i
n
 
n
o
d
e
_
d
a
t
a
:

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
{
n
o
d
e
_
i
d
}
:
 
{
n
o
d
e
_
d
a
t
a
[
'
e
r
r
o
r
'
]
}
"
)

 
 
 
 
e
l
s
e
:

 
 
 
 
 
 
 
 
l
a
t
e
s
t
_
r
o
u
n
d
 
=
 
n
o
d
e
_
d
a
t
a
.
g
e
t
(
'
l
a
t
e
s
t
_
r
o
u
n
d
'
,
 
'
N
/
A
'
)

 
 
 
 
 
 
 
 
e
l
a
p
s
e
d
_
s
e
c
 
=
 
n
o
d
e
_
d
a
t
a
.
g
e
t
(
'
e
l
a
p
s
e
d
_
s
e
c
o
n
d
s
'
,
 
0
)

 
 
 
 
 
 
 
 
e
l
a
p
s
e
d
_
h
o
u
r
s
 
=
 
e
l
a
p
s
e
d
_
s
e
c
 
/
 
3
6
0
0

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
{
n
o
d
e
_
i
d
}
:
 
R
o
u
n
d
 
{
l
a
t
e
s
t
_
r
o
u
n
d
}
 
|
 
{
e
l
a
p
s
e
d
_
h
o
u
r
s
:
.
1
f
}
h
 
e
l
a
p
s
e
d
"
)


p
r
i
n
t
(
)

p
r
i
n
t
(
"
=
"
*
7
0
)

p
r
i
n
t
(
"
N
o
t
e
:
 
P
r
o
g
r
e
s
s
 
u
p
d
a
t
e
s
 
e
v
e
r
y
 
r
o
u
n
d
.
 
L
o
g
s
 
f
l
u
s
h
 
e
v
e
r
y
 
3
0
s
 
t
o
 
G
D
r
i
v
e
.
"
)

p
r
i
n
t
(
"
W
a
t
c
h
 
a
d
a
p
t
i
v
e
 
I
/
J
 
v
a
l
u
e
s
 
e
v
o
l
v
e
 
i
n
 
C
e
l
l
 
8
 
a
n
a
l
y
s
i
s
!
"
)

## 8. Analyze Adaptive Behavior

**See how I/J evolved during training!**

In [ ]:
from rgym_exp.utils.experiment_manager import get_experiment_metrics
import matplotlib.pyplot as plt
import numpy as np

print("="*70)
print(f"Adaptive I/J Results: {EXPERIMENT_NAME}")
print("="*70)
print(f"Model: {MODEL_NAME}")
print(f"Rounds: {MAX_ROUNDS}")
print(f"Initial: I={INITIAL_I}, J={INITIAL_J}")
print(f"Adaptation Rate: α={ADAPTATION_RATE}")
print()

df = get_experiment_metrics(GDRIVE_BASE_PATH, EXPERIMENT_NAME)

if not df.empty and 'adaptive_J' in df.columns:
    # Aggregate by round (take last value per round)
    round_df = df.groupby('round').last().reset_index()
    
    final_I = int(round_df['adaptive_I'].iloc[-1])
    final_J = int(round_df['adaptive_J'].iloc[-1])
    final_J_cont = round_df['adaptive_J_continuous'].iloc[-1]
    
    total_reward = df['my_reward'].sum()
    
    print("Final Adaptive State:")
    print(f"  I={final_I}, J={final_J} (J_continuous={final_J_cont:.2f})")
    print(f"  Total Reward: {total_reward:.2f}")
    print()
    
    # Statistics
    mean_J = round_df['adaptive_J'].mean()
    std_J = round_df['adaptive_J'].std()
    
    print("Adaptive Statistics:")
    print(f"  Mean J: {mean_J:.2f} ± {std_J:.2f}")
    print(f"  Started at: J={INITIAL_J}")
    print(f"  Ended at: J={final_J}")
    print(f"  Change: {final_J - INITIAL_J:+d} ({(final_J - INITIAL_J) / INITIAL_J * 100:+.0f}%)")
    print()
    
    # Compare to baseline (if available)
    print("Comparison to Fixed Ratios:")
    print("  (Based on SAPO paper expectations)")
    print(f"  Config 1 (I=3, J=1): +52% improvement")
    print(f"  Config 2 (I=2, J=2): +94% improvement (BEST fixed)")
    print(f"  Config 3 (I=1, J=3): +68% improvement")
    print(f"  Adaptive (I={final_I}, J={final_J}): TBD (run baseline first)")
    print()
    
    # Plot adaptive trajectory
    if MAX_ROUNDS >= 20:
        fig, axes = plt.subplots(2, 1, figsize=(12, 10))
        
        # Plot 1: J evolution over time
        ax1 = axes[0]
        ax1.plot(round_df['round'], round_df['adaptive_J_continuous'], 
                label='J (continuous)', linewidth=2, alpha=0.7)
        ax1.plot(round_df['round'], round_df['adaptive_J'], 
                label='J (discrete)', marker='o', markersize=3, alpha=0.5)
        ax1.axhline(y=INITIAL_J, color='gray', linestyle='--', 
                   label=f'Initial J={INITIAL_J}')
        ax1.set_xlabel('Round')
        ax1.set_ylabel('J (External Rollouts)')
        ax1.set_title('Adaptive I/J Evolution Over Training', fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Cumulative rewards
        ax2 = axes[1]
        for node_id in df['node_id'].unique():
            node_df = df[df['node_id'] == node_id].sort_values('round')
            ax2.plot(node_df['round'], node_df['my_reward'].cumsum(), 
                    label=node_id, alpha=0.7)
        ax2.set_xlabel('Round')
        ax2.set_ylabel('Cumulative Reward')
        ax2.set_title('Cumulative Rewards with Adaptive I/J', fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plot_path = f'{GDRIVE_BASE_PATH}/adaptive_ij_results.png'
        plt.savefig(plot_path, dpi=150, bbox_inches='tight')
        print(f"✓ Plot saved: {plot_path}")
        plt.show()
        
        # Interpretation
        print()
        print("="*70)
        print("INTERPRETATION:")
        print("="*70)
        if final_J > INITIAL_J + 0.5:
            print("✅ Algorithm increased J → Swarm sharing is beneficial!")
            print("   External rollouts help more than local ones.")
        elif final_J < INITIAL_J - 0.5:
            print("⚠️  Algorithm decreased J → Local training preferred")
            print("   More benefit from own rollouts than swarm.")
        else:
            print("➡️  Algorithm kept J near initial → Balanced split optimal")
            print("   Local and external rollouts equally valuable.")
        print()
        
        if final_J >= 2:
            print("📊 THESIS INSIGHT:")
            print("   Adaptive algorithm discovered swarm collaboration value!")
            print("   This validates the need for external rollouts (J > 0).")
        else:
            print("📊 THESIS INSIGHT:")
            print("   Adaptive algorithm favored local training.")
            print("   May indicate: model too weak, swarm too small, or")
            print("   early training phase benefits from focused exploration.")
else:
    print("❌ No adaptive metrics found - verify ADAPTIVE_IJ_ENABLED=True")

In [ ]:
i
m
p
o
r
t
 
s
h
u
t
i
l

i
m
p
o
r
t
 
o
s

f
r
o
m
 
d
a
t
e
t
i
m
e
 
i
m
p
o
r
t
 
d
a
t
e
t
i
m
e


p
r
i
n
t
(
"
=
"
*
7
0
)

p
r
i
n
t
(
"
S
A
V
I
N
G
 
E
X
P
E
R
I
M
E
N
T
 
R
E
S
U
L
T
S
 
T
O
 
G
O
O
G
L
E
 
D
R
I
V
E
"
)

p
r
i
n
t
(
"
=
"
*
7
0
)

p
r
i
n
t
(
)


#
 
P
a
t
h
s

e
x
p
e
r
i
m
e
n
t
_
p
a
t
h
 
=
 
f
"
{
G
D
R
I
V
E
_
B
A
S
E
_
P
A
T
H
}
/
e
x
p
e
r
i
m
e
n
t
s
/
{
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
}
"

a
r
c
h
i
v
e
_
d
i
r
 
=
 
f
"
{
G
D
R
I
V
E
_
B
A
S
E
_
P
A
T
H
}
/
e
x
p
e
r
i
m
e
n
t
_
a
r
c
h
i
v
e
s
"

o
s
.
m
a
k
e
d
i
r
s
(
a
r
c
h
i
v
e
_
d
i
r
,
 
e
x
i
s
t
_
o
k
=
T
r
u
e
)


#
 
C
r
e
a
t
e
 
t
i
m
e
s
t
a
m
p
 
f
o
r
 
a
r
c
h
i
v
e
 
n
a
m
e

t
i
m
e
s
t
a
m
p
 
=
 
d
a
t
e
t
i
m
e
.
n
o
w
(
)
.
s
t
r
f
t
i
m
e
(
"
%
Y
%
m
%
d
_
%
H
%
M
%
S
"
)

a
r
c
h
i
v
e
_
n
a
m
e
 
=
 
f
"
{
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
}
_
{
t
i
m
e
s
t
a
m
p
}
"

a
r
c
h
i
v
e
_
p
a
t
h
 
=
 
f
"
{
a
r
c
h
i
v
e
_
d
i
r
}
/
{
a
r
c
h
i
v
e
_
n
a
m
e
}
"


p
r
i
n
t
(
f
"
E
x
p
e
r
i
m
e
n
t
:
 
{
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
}
"
)

p
r
i
n
t
(
f
"
S
o
u
r
c
e
:
 
{
e
x
p
e
r
i
m
e
n
t
_
p
a
t
h
}
"
)

p
r
i
n
t
(
f
"
A
r
c
h
i
v
e
:
 
{
a
r
c
h
i
v
e
_
p
a
t
h
}
.
z
i
p
"
)

p
r
i
n
t
(
)


#
 
C
h
e
c
k
 
i
f
 
e
x
p
e
r
i
m
e
n
t
 
e
x
i
s
t
s

i
f
 
n
o
t
 
o
s
.
p
a
t
h
.
e
x
i
s
t
s
(
e
x
p
e
r
i
m
e
n
t
_
p
a
t
h
)
:

 
 
 
 
p
r
i
n
t
(
"
❌
 
E
R
R
O
R
:
 
E
x
p
e
r
i
m
e
n
t
 
d
i
r
e
c
t
o
r
y
 
n
o
t
 
f
o
u
n
d
!
"
)

 
 
 
 
p
r
i
n
t
(
f
"
 
 
 
L
o
o
k
i
n
g
 
f
o
r
:
 
{
e
x
p
e
r
i
m
e
n
t
_
p
a
t
h
}
"
)

 
 
 
 
p
r
i
n
t
(
"
 
 
 
M
a
k
e
 
s
u
r
e
 
t
r
a
i
n
i
n
g
 
h
a
s
 
r
u
n
 
a
n
d
 
c
r
e
a
t
e
d
 
t
h
e
 
e
x
p
e
r
i
m
e
n
t
 
d
i
r
e
c
t
o
r
y
.
"
)

e
l
s
e
:

 
 
 
 
t
r
y
:

 
 
 
 
 
 
 
 
#
 
C
o
u
n
t
 
f
i
l
e
s
 
t
o
 
a
r
c
h
i
v
e

 
 
 
 
 
 
 
 
t
o
t
a
l
_
f
i
l
e
s
 
=
 
s
u
m
(
l
e
n
(
f
i
l
e
s
)
 
f
o
r
 
_
,
 
_
,
 
f
i
l
e
s
 
i
n
 
o
s
.
w
a
l
k
(
e
x
p
e
r
i
m
e
n
t
_
p
a
t
h
)
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
F
o
u
n
d
 
{
t
o
t
a
l
_
f
i
l
e
s
}
 
f
i
l
e
s
 
t
o
 
a
r
c
h
i
v
e
.
.
.
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
)

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
#
 
C
r
e
a
t
e
 
t
h
e
 
a
r
c
h
i
v
e

 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
C
r
e
a
t
i
n
g
 
a
r
c
h
i
v
e
.
.
.
"
)

 
 
 
 
 
 
 
 
s
h
u
t
i
l
.
m
a
k
e
_
a
r
c
h
i
v
e
(
a
r
c
h
i
v
e
_
p
a
t
h
,
 
'
z
i
p
'
,
 
e
x
p
e
r
i
m
e
n
t
_
p
a
t
h
)

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
#
 
G
e
t
 
a
r
c
h
i
v
e
 
s
i
z
e

 
 
 
 
 
 
 
 
a
r
c
h
i
v
e
_
s
i
z
e
 
=
 
o
s
.
p
a
t
h
.
g
e
t
s
i
z
e
(
f
"
{
a
r
c
h
i
v
e
_
p
a
t
h
}
.
z
i
p
"
)
 
/
 
(
1
0
2
4
 
*
 
1
0
2
4
)
 
 
#
 
M
B

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
✅
 
A
r
c
h
i
v
e
 
c
r
e
a
t
e
d
 
s
u
c
c
e
s
s
f
u
l
l
y
!
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
 
S
i
z
e
:
 
{
a
r
c
h
i
v
e
_
s
i
z
e
:
.
1
f
}
 
M
B
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
)

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
#
 
A
l
s
o
 
s
a
v
e
 
a
 
s
u
m
m
a
r
y
 
f
i
l
e

 
 
 
 
 
 
 
 
s
u
m
m
a
r
y
_
p
a
t
h
 
=
 
f
"
{
a
r
c
h
i
v
e
_
d
i
r
}
/
{
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
}
_
s
u
m
m
a
r
y
.
t
x
t
"

 
 
 
 
 
 
 
 
w
i
t
h
 
o
p
e
n
(
s
u
m
m
a
r
y
_
p
a
t
h
,
 
'
w
'
)
 
a
s
 
f
:

 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
E
x
p
e
r
i
m
e
n
t
:
 
{
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
}
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
M
o
d
e
l
:
 
{
M
O
D
E
L
_
N
A
M
E
}
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
C
o
n
f
i
g
u
r
a
t
i
o
n
:
 
A
d
a
p
t
i
v
e
 
I
/
J
 
(
s
t
a
r
t
e
d
 
w
i
t
h
 
I
=
{
I
N
I
T
I
A
L
_
I
}
,
 
J
=
{
I
N
I
T
I
A
L
_
J
}
)
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
T
o
t
a
l
 
S
a
m
p
l
e
s
:
 
{
T
O
T
A
L
_
S
A
M
P
L
E
S
}
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
A
d
a
p
t
a
t
i
o
n
 
R
a
t
e
:
 
{
A
D
A
P
T
A
T
I
O
N
_
R
A
T
E
}
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
M
a
x
 
R
o
u
n
d
s
:
 
{
M
A
X
_
R
O
U
N
D
S
}
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
N
o
d
e
s
:
 
{
N
U
M
_
N
O
D
E
S
}
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
D
a
t
e
:
 
{
t
i
m
e
s
t
a
m
p
}
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
A
r
c
h
i
v
e
:
 
{
a
r
c
h
i
v
e
_
n
a
m
e
}
.
z
i
p
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
A
r
c
h
i
v
e
 
S
i
z
e
:
 
{
a
r
c
h
i
v
e
_
s
i
z
e
:
.
1
f
}
 
M
B
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
#
 
T
r
y
 
t
o
 
a
d
d
 
f
i
n
a
l
 
m
e
t
r
i
c
s

 
 
 
 
 
 
 
 
 
 
 
 
t
r
y
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
r
o
m
 
r
g
y
m
_
e
x
p
.
u
t
i
l
s
.
e
x
p
e
r
i
m
e
n
t
_
m
a
n
a
g
e
r
 
i
m
p
o
r
t
 
g
e
t
_
e
x
p
e
r
i
m
e
n
t
_
s
t
a
t
u
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
t
a
t
u
s
 
=
 
g
e
t
_
e
x
p
e
r
i
m
e
n
t
_
s
t
a
t
u
s
(
G
D
R
I
V
E
_
B
A
S
E
_
P
A
T
H
,
 
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
s
t
a
t
u
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
\
n
F
i
n
a
l
 
M
e
t
r
i
c
s
:
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
R
o
u
n
d
s
 
C
o
m
p
l
e
t
e
d
:
 
{
s
t
a
t
u
s
.
g
e
t
(
'
c
u
r
r
e
n
t
_
r
o
u
n
d
'
,
 
'
N
/
A
'
)
}
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
T
o
t
a
l
 
C
u
m
u
l
a
t
i
v
e
 
R
e
w
a
r
d
:
 
{
s
t
a
t
u
s
.
g
e
t
(
'
t
o
t
a
l
_
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
'
,
 
'
N
/
A
'
)
}
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
T
r
y
 
t
o
 
g
e
t
 
f
i
n
a
l
 
I
/
J
 
v
a
l
u
e
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
t
r
y
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
r
o
m
 
r
g
y
m
_
e
x
p
.
u
t
i
l
s
.
e
x
p
e
r
i
m
e
n
t
_
m
a
n
a
g
e
r
 
i
m
p
o
r
t
 
g
e
t
_
e
x
p
e
r
i
m
e
n
t
_
m
e
t
r
i
c
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
d
f
 
=
 
g
e
t
_
e
x
p
e
r
i
m
e
n
t
_
m
e
t
r
i
c
s
(
G
D
R
I
V
E
_
B
A
S
E
_
P
A
T
H
,
 
E
X
P
E
R
I
M
E
N
T
_
N
A
M
E
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
d
f
.
e
m
p
t
y
 
a
n
d
 
'
a
d
a
p
t
i
v
e
_
J
'
 
i
n
 
d
f
.
c
o
l
u
m
n
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
i
n
a
l
_
I
 
=
 
i
n
t
(
d
f
[
'
a
d
a
p
t
i
v
e
_
I
'
]
.
i
l
o
c
[
-
1
]
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
i
n
a
l
_
J
 
=
 
i
n
t
(
d
f
[
'
a
d
a
p
t
i
v
e
_
J
'
]
.
i
l
o
c
[
-
1
]
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
F
i
n
a
l
 
A
d
a
p
t
i
v
e
 
V
a
l
u
e
s
:
 
I
=
{
f
i
n
a
l
_
I
}
,
 
J
=
{
f
i
n
a
l
_
J
}
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
x
c
e
p
t
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
a
s
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
#
 
P
e
r
-
n
o
d
e
 
r
e
w
a
r
d
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
 
=
 
s
t
a
t
u
s
.
g
e
t
(
'
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
'
,
 
{
}
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
\
n
P
e
r
-
N
o
d
e
 
C
u
m
u
l
a
t
i
v
e
 
R
e
w
a
r
d
s
:
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
n
o
d
e
_
i
d
 
i
n
 
s
o
r
t
e
d
(
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
.
k
e
y
s
(
)
)
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
.
w
r
i
t
e
(
f
"
 
 
{
n
o
d
e
_
i
d
}
:
 
{
c
u
m
u
l
a
t
i
v
e
_
r
e
w
a
r
d
s
[
n
o
d
e
_
i
d
]
:
.
2
f
}
\
n
"
)

 
 
 
 
 
 
 
 
 
 
 
 
e
x
c
e
p
t
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
a
s
s

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
📊
 
S
u
m
m
a
r
y
 
s
a
v
e
d
 
t
o
:
"
,
 
s
u
m
m
a
r
y
_
p
a
t
h
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
)

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
#
 
L
i
s
t
 
i
m
p
o
r
t
a
n
t
 
f
i
l
e
s
 
f
o
r
 
q
u
i
c
k
 
a
c
c
e
s
s

 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
K
e
y
 
F
i
l
e
s
 
S
a
v
e
d
:
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
•
 
L
o
g
s
:
 
{
e
x
p
e
r
i
m
e
n
t
_
p
a
t
h
}
/
l
o
g
s
/
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
•
 
C
h
e
c
k
p
o
i
n
t
s
:
 
{
e
x
p
e
r
i
m
e
n
t
_
p
a
t
h
}
/
c
h
e
c
k
p
o
i
n
t
s
/
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
•
 
R
e
w
a
r
d
s
:
 
{
e
x
p
e
r
i
m
e
n
t
_
p
a
t
h
}
/
r
e
w
a
r
d
s
/
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
•
 
P
r
o
g
r
e
s
s
:
 
{
e
x
p
e
r
i
m
e
n
t
_
p
a
t
h
}
/
p
r
o
g
r
e
s
s
_
*
.
j
s
o
n
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
)

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
#
 
D
o
w
n
l
o
a
d
 
l
i
n
k
 
(
f
o
r
 
C
o
l
a
b
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
📥
 
T
o
 
d
o
w
n
l
o
a
d
 
t
h
e
 
a
r
c
h
i
v
e
:
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
 
1
.
 
N
a
v
i
g
a
t
e
 
t
o
 
G
o
o
g
l
e
 
D
r
i
v
e
:
 
{
a
r
c
h
i
v
e
_
d
i
r
}
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
 
2
.
 
D
o
w
n
l
o
a
d
:
 
{
a
r
c
h
i
v
e
_
n
a
m
e
}
.
z
i
p
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
)

 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
#
 
O
p
t
i
o
n
 
t
o
 
c
o
p
y
 
t
o
 
/
c
o
n
t
e
n
t
 
f
o
r
 
i
m
m
e
d
i
a
t
e
 
d
o
w
n
l
o
a
d

 
 
 
 
 
 
 
 
c
o
p
y
_
t
o
_
c
o
n
t
e
n
t
 
=
 
i
n
p
u
t
(
"
C
o
p
y
 
a
r
c
h
i
v
e
 
t
o
 
/
c
o
n
t
e
n
t
 
f
o
r
 
i
m
m
e
d
i
a
t
e
 
d
o
w
n
l
o
a
d
?
 
(
y
/
n
)
:
 
"
)

 
 
 
 
 
 
 
 
i
f
 
c
o
p
y
_
t
o
_
c
o
n
t
e
n
t
.
l
o
w
e
r
(
)
 
=
=
 
'
y
'
:

 
 
 
 
 
 
 
 
 
 
 
 
c
o
n
t
e
n
t
_
p
a
t
h
 
=
 
f
"
/
c
o
n
t
e
n
t
/
{
a
r
c
h
i
v
e
_
n
a
m
e
}
.
z
i
p
"

 
 
 
 
 
 
 
 
 
 
 
 
s
h
u
t
i
l
.
c
o
p
y
(
f
"
{
a
r
c
h
i
v
e
_
p
a
t
h
}
.
z
i
p
"
,
 
c
o
n
t
e
n
t
_
p
a
t
h
)

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
✅
 
A
r
c
h
i
v
e
 
c
o
p
i
e
d
 
t
o
:
 
{
c
o
n
t
e
n
t
_
p
a
t
h
}
"
)

 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
 
 
 
Y
o
u
 
c
a
n
 
n
o
w
 
d
o
w
n
l
o
a
d
 
i
t
 
f
r
o
m
 
t
h
e
 
F
i
l
e
s
 
p
a
n
e
l
 
i
n
 
C
o
l
a
b
"
)

 
 
 
 
 
 
 
 
 
 
 
 

 
 
 
 
 
 
 
 
 
 
 
 
#
 
T
r
y
 
t
o
 
t
r
i
g
g
e
r
 
d
o
w
n
l
o
a
d
 
i
n
 
C
o
l
a
b

 
 
 
 
 
 
 
 
 
 
 
 
t
r
y
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
r
o
m
 
g
o
o
g
l
e
.
c
o
l
a
b
 
i
m
p
o
r
t
 
f
i
l
e
s

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
i
l
e
s
.
d
o
w
n
l
o
a
d
(
c
o
n
t
e
n
t
_
p
a
t
h
)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
📥
 
D
o
w
n
l
o
a
d
 
s
t
a
r
t
e
d
.
.
.
"
)

 
 
 
 
 
 
 
 
 
 
 
 
e
x
c
e
p
t
:

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
 
 
 
(
A
u
t
o
m
a
t
i
c
 
d
o
w
n
l
o
a
d
 
n
o
t
 
a
v
a
i
l
a
b
l
e
 
-
 
u
s
e
 
F
i
l
e
s
 
p
a
n
e
l
)
"
)

 
 
 
 
 
 
 
 

 
 
 
 
e
x
c
e
p
t
 
E
x
c
e
p
t
i
o
n
 
a
s
 
e
:

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
❌
 
E
R
R
O
R
 
c
r
e
a
t
i
n
g
 
a
r
c
h
i
v
e
:
 
{
e
}
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
"
 
 
 
T
h
e
 
e
x
p
e
r
i
m
e
n
t
 
d
a
t
a
 
i
s
 
s
t
i
l
l
 
i
n
 
G
o
o
g
l
e
 
D
r
i
v
e
 
a
t
:
"
)

 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
"
 
 
 
{
e
x
p
e
r
i
m
e
n
t
_
p
a
t
h
}
"
)


p
r
i
n
t
(
)

p
r
i
n
t
(
"
=
"
*
7
0
)

p
r
i
n
t
(
"
D
O
N
E
!
"
)

p
r
i
n
t
(
"
=
"
*
7
0
)

## 9. Archive Results to Google Drive (NEW!)

Run this cell after training completes to save all results to Google Drive.
This creates a compressed archive of the entire experiment for easy download and backup.